In [ ]:
# 下記記事を写経
# 参考:https://qiita.com/tomov3/items/039d4271ed30490edf7b
# 参考2:https://teratail.com/questions/137370
# 「Python ではじめる機械学習」5章

### train_test_split() により得られた training_set に依存したスコアを算出

In [1]:
# 必要なライブラリの import
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# データのロード
iris = load_iris()

In [3]:
# データの分割
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)

In [6]:
# モデル宣言
model = LogisticRegression()

In [12]:
# training set を用いて学習
lr = model.fit(X_train, y_train)

In [13]:
# test set を用いて評価
score = lr.score(X_test, y_test)
print('Test set score: {}'.format(score))

Test set score: 0.868421052631579


## 交差検証
データセットをk個に分割し，モデルの訓練と評価をk回実行
得られたk個の評価値の平均をとった値を最終的なモデルのスコアとする

In [9]:
from sklearn.model_selection import cross_val_score

In [23]:
# 交差検証
# cvのデフォルトは3
scores = cross_val_score(model, iris.data, iris.target, cv=5)

In [24]:
# 各分割におけるスコア
# parameter:https://qiita.com/kibinag0/items/1a29db61fcb8c527d952
print('Cross_Validation_scores: {}'.format(scores))

Cross_Validation_scores: [1.         0.96666667 0.93333333 0.9        1.        ]


In [25]:
# スコアの平均値
import numpy as np
print('Average_score: {}'.format(np.mean(scores)))

Average_score: 0.9600000000000002


In [27]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [28]:
# 単純なk分割交差検証
# データの最初から1/kずつ分割
kfold = KFold(n_splits=3)
print('Cross_validation_scores: \n{}'.format(cross_val_score(model, iris.data, iris.target, cv=kfold)))

Cross_validation_scores: 
[0. 0. 0.]


In [29]:
# 層化k分割交差検証 
# 各分割内でのクラスの比率が全体の比率と同じになるように分割
stratifiedkfold = StratifiedKFold(n_splits=3)
print('Cross_validation_scores: \n{}'.format(cross_val_score(model, iris.data, iris.target, cv=stratifiedkfold)))

Cross_validation_scores: 
[0.96078431 0.92156863 0.95833333]


In [ ]:
# 一般的には，回帰には単純なk分割交差検証，クラス分類には層化k分割交差検証が用いられる
# cross_val_score()のパラメータcvに何も指定しない場合はこの選択基準で分割方法が選択される

### ハイパーパラメータのチューニング
単純なグリッドサーチ

In [31]:
from sklearn.svm import SVC

In [32]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)

In [35]:
param_list = [0.001, 0.01, 0.1, 1, 10, 100]

In [54]:
best_score = 0
best_parameters = {}

for gamma in param_list:
    for C in param_list:
        model = SVC(gamma=gamma, C=C)
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        # 最も良いスコアのパラメータとスコアを更新
        if score > best_score:
            best_score = score
            best_parameters = {'gamma' : gamma, 'C' : C}

In [55]:
print('Best_score: {}'.format(best_score))
print('Best_parameters: {}'.format(best_parameters))

Best_score: 0.9736842105263158
Best_parameters: {'gamma': 0.001, 'C': 100}


上記単純なグリッドサーチのコードはパラメータ選択の際にtest_setを使用してしまいっていることが問題（score = svm.score(X_test, y_test) の部分）
test_setは本来training_setを用いて学習が完了したモデルの汎化精度を評価するためのデータであることから、
training_setに分割した後にさらにデータの分割を行い、パラメータ選択のためのデータセットvalidation_setを新たに作成する必要あり

In [39]:
# X_trainval = X_train + X_valid 

# irisデータを(訓練セット+検証セット)とテストセットに分割する
## この(訓練セット+検証セット)を後で訓練セットと検証セットに分けるための準備
X_trainval, X_test, y_trainval, y_test = train_test_split(iris.data, iris.target, random_state=0)
# 作成した(訓練セット+検証セット)をさらに訓練セットと検証セットに分割する
X_train, X_valid, y_train, y_valid = train_test_split(X_trainval, y_trainval, random_state=1)

In [50]:
print(X_trainval.shape)
print(X_test.shape)
print(y_trainval.shape)
print(y_test.shape)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(112, 4)
(38, 4)
(112,)
(38,)
(84, 4)
(28, 4)
(84,)
(28,)


In [57]:
best_score = 0
best_parameters = {}
param_list = [0.001, 0.01, 0.1, 1, 10, 100]

for gamma in param_list:
    for C in param_list:
        model = SVC(gamma=gamma, C=C)
        model.fit(X_train, y_train)
        # validation_setを用いてscoreを計算する
        score = model.score(X_valid, y_valid)
        if score > best_score:
            best_score = score
            best_parameters = {'gamma' : gamma, 'C' : C}

In [42]:
# **kwargs: 複数のキーワード引数を辞書として受け取る
# 参考:https://note.nkmk.me/python-args-kwargs-usage/
model = SVC(**best_parameters)

In [51]:
# best_parametersに対しtraining_set + validation_set を用いて学習
model.fit(X_trainval, y_trainval)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [52]:
# test_setによる評価はbest_parametersが得られて初めて実行される
test_score = model.score(X_test, y_test)

In [53]:
print('Best score on validation set: {}'.format(best_score))
print('Best parameters: {}'.format(best_parameters))
print('Test set score with best parameters: {}'.format(test_score))

Best score on validation set: 0.9642857142857143
Best parameters: {'gamma': 0.001, 'C': 10}
Test set score with best parameters: 0.9210526315789473


## グリッドサーチと交差検証の組み合わせ
各パラメータの組合わせにおける評価値の計算の際に交差検証を実行

In [58]:
best_score = 0
best_parameters  = {}

for gamma in param_list:
    for C in param_list:
        model = SVC(gamma=gamma, C=C)
        # cross_val_score() による交差検証
        scores = cross_val_score(model, X_trainval, y_trainval, cv=5)
        # k 個の評価値の平均を用いる
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_parameters = {'gamma' : gamma, 'C' : C}

In [59]:
# モデル宣言
model = SVC(**best_parameters)

In [60]:
# best_parametersに対し、training_set + validation_setを用いて学習
model.fit(X_trainval, y_trainval)
# test_setによる評価は、best_parameters が得られて初めて実行される
test_score = model.score(X_test, y_test)

In [61]:
print('Best score on validation set: {}'.format(best_score))
print('Best parameters: {}'.format(best_parameters))
print('Test set score with best parameters: {}'.format(test_score))

Best score on validation set: 0.9726896292113683
Best parameters: {'gamma': 0.01, 'C': 100}
Test set score with best parameters: 0.9736842105263158


# 交差検証を用いたグリッドサーチを実装したGridSearchCV
### GridSearchCV関数はハイパーパラメータの探索、モデル学習、交差検定での評価をすべて行う
GridSearchCVを用いることで上記のコードを下記のように書き直すことが可能

In [62]:
from sklearn.model_selection import GridSearchCV

In [63]:
# パラメータを辞書型で指定
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],  'gamma' : [0.001, 0.01, 0.1, 1, 10, 100]}

In [64]:
# validation_set は GridSearchCV によって自動作成されるためtraining_setとtest_set の分割のみを実行すればよい
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)

In [65]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(112, 4)
(38, 4)
(112,)
(38,)


In [67]:
# モデル宣言(GridSerch使用)
model = GridSearchCV(SVC(), param_grid, cv=5)

In [68]:
# fit関数を呼ぶことで交差検証とグリッドサーチがどちらも実行される
model.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [71]:
print('Test set score : {:.2f}'.format(model.score(X_test, y_test)))

Test set score : 0.97
